In [1]:
import dill
from Bio import SeqIO
from Bio.Seq import Seq
from freqs import Freqs, load_freqs, dump_freqs

In [12]:
%load_ext autoreload
%aimport freqs
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## モジュールの動作確認&pressureの算出（02/04）

In [29]:
cds_fp = "./data/GCF_900128725.1_BCifornacula_v1.0_cds_from_genomic.fna"
cdss = list(SeqIO.parse(cds_fp, 'fasta'))
freqs = Freqs()
for cds in cdss:
    freqs.update(cds.seq)

length error: not multiple of 3 (1357)
length error: not multiple of 3 (623)
stop error: inframe stop codon found
length error: not multiple of 3 (947)
length error: not multiple of 3 (407)
length error: not multiple of 3 (812)


## freqsからそれぞれのcodonの確率を算出（02/04）

In [9]:
freqs_fp = "./data/freqs/GCF_000010525.1.freqs"
freqs = load_freqs(freqs_fp)

In [10]:
freqs.aa['*']

4723

In [4]:
import itertools

In [11]:
codons = [''.join(_) for _ in itertools.product("TCAG", repeat=3)]

probs = dict()
for codon in codons:
            aa = str(Seq(codon).translate(table=11))
            probs[codon] = freqs.codon[codon] / freqs.aa[aa] if freqs.aa[aa] > 0 else 0
probs

{'AAA': 0.07672873788522269,
 'AAC': 0.6730963359070199,
 'AAG': 0.9232712621147773,
 'AAT': 0.3269036640929801,
 'ACA': 0.033610214709274555,
 'ACC': 0.5788878921677132,
 'ACG': 0.36385241795496115,
 'ACT': 0.023649475168051073,
 'AGA': 0.006185030251889621,
 'AGC': 0.2948471999902247,
 'AGG': 0.02073645846424532,
 'AGT': 0.02265423575556886,
 'ATA': 0.012407319229984354,
 'ATC': 0.893612679409564,
 'ATG': 1.0,
 'ATT': 0.09398000136045168,
 'CAA': 0.07281771983618157,
 'CAC': 0.5645443443245206,
 'CAG': 0.9271822801638184,
 'CAT': 0.43545565567547945,
 'CCA': 0.029226178272853596,
 'CCC': 0.401273377525684,
 'CCG': 0.5217772614410351,
 'CCT': 0.04772318276042734,
 'CGA': 0.020942925366703172,
 'CGC': 0.627031005942656,
 'CGG': 0.2481821935761863,
 'CGT': 0.07692238639831954,
 'CTA': 0.006056307289393278,
 'CTC': 0.40847640525728696,
 'CTG': 0.47519278335515786,
 'CTT': 0.0736092924002134,
 'GAA': 0.2870973880774537,
 'GAC': 0.6791865108000192,
 'GAG': 0.7129026119225462,
 'GAT': 0.320

In [12]:
dct_lst = []
for c1 in codons:
    for c2 in codons:
        bicodon = c1 + c2
        biaa = str(Seq(bicodon).translate(table=11))
        a1 = biaa[0]
        a2 = biaa[1]
        
        dct = {
            "bicodon": bicodon,
            "freq": freqs.bicodon[c1][c2],
            "pred": freqs.biaa[a1][a2] * probs[c1] * probs[c2]
        }
        dct_lst.append(dct)

In [13]:
pres_df = pd.DataFrame(dct_lst)
pres_df["pres"] = pres_df["freq"] / pres_df["pred"]
pres_df[pres_df["pres"] > 0].sort_values(by="pres")

,bicodon,freq,pred,pres
2603,AATAAG,18,281.296947,0.063989
1069,CTTAGC,15,232.205680,0.064598
2083,ATTATG,7,108.358942,0.064600
179,TTAGTG,1,15.275552,0.065464
2093,ATTAGC,8,120.675919,0.066293
3529,GCGTAC,43,606.655677,0.070880
3528,GCGTAT,70,959.834314,0.072929
456,TCGTAT,25,316.367053,0.079022
2091,ATTAAG,13,155.490110,0.083607
2504,ACGTAT,39,454.134966,0.085878


## cdsファイルから各frequenciesを算出する（02/03）

In [2]:
class Freqs:
    def __init__(self):
        self.codon = Counter()
        self.aa = Counter()
        self.bicodon = defaultdict(lambda: Counter())
        self.biaa = defaultdict(lambda: Counter())
        
    def _is_typical(self, cds_seq):
        """
        Check the following:
        * length is multiple of 3
        * Only ATGC used
        * No premature stop codon included
        """
        
        if len(cds_seq) % 3 != 0:
            print("length error")
            return False
        
        if not(set(cds_seq) <= set("ATGC")):
            print("character error: unknown characters ({}) included".format(','.join(set(cds_seq) - set("ATGC"))))
            return False
        
        pro = cds_seq.translate(table=11)
        if "*" in pro[:-1]:
            print("stop error: inframe stop codon found")
            return False
        
        return True
        
    def update(self, cds_seq):
        """
        Args:
            cds_seq: Bio.Seq object
        """
        
        if not(self._is_typical(cds_seq)):
            return False
    
        pro_seq = cds_seq.translate(table=11)
        codons = [str(cds_seq[3*i:3*(i+1)]) for i in range(int(len(cds_seq)/3))]
        aas = list(cds.seq.translate())
        
        self.codon.update(codons)
        self.aa.update(aas)
        for i in range(len(codons)-1):
            self.bicodon[codons[i]][codons[i+1]] += 1
        for i in range(len(aas) - 1):
            self.biaa[aas[i]][aas[i+1]] += 1

        return True
    

def dump_freqs(freqs, fp):
    dill.dump(freqs, open(fp, "wb"))
    
def load_freqs(fp):
    return dill.load(open(fp, "rb"))

In [3]:
cds_fp = "./data/GCF_900128725.1_BCifornacula_v1.0_cds_from_genomic.fna"
cdss = list(SeqIO.parse(cds_fp, 'fasta'))
len(cdss)

383

In [4]:
freqs = Freqs()
for cds in cdss:
    freqs.update(cds.seq)

length error
length error
stop error: inframe stop codon found
length error
length error
length error


In [6]:
fp = "tmp.freqs"
dump_freqs(freqs, fp)
freqs_new = load_freqs(fp)